# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,994.908298,-0.789087,1234,994.119211,995.697386,1988.238422,1991.394771
6,397.318083,-0.928803,1234,396.389280,398.246886,792.778561,796.493772
7,63.093114,-0.234675,1234,62.858439,63.327789,125.716878,126.655578
8,797.961161,-0.096376,1234,797.864784,798.057537,1595.729569,1596.115073
9,517.850160,0.990924,1234,518.841084,516.859235,1037.682169,1033.718471
10,551.501534,-0.375255,1234,551.126279,551.876789,1102.252558,1103.753578
11,124.955579,-0.323829,1234,124.631750,125.279408,249.263500,250.558815
12,624.892276,-0.951232,1234,623.941045,625.843508,1247.882089,1251.687016
13,319.927760,0.747405,1234,320.675165,319.180355,641.350330,638.360711
14,332.125163,-0.643791,1234,331.481372,332.768955,662.962744,665.537909


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-994.119211,995.697386
6,-396.389280,398.246886
7,-62.858439,63.327789
8,-797.864784,798.057537
9,-518.841084,516.859235
10,-551.126279,551.876789
11,-124.631750,125.279408
12,-623.941045,625.843508
13,-320.675165,319.180355
14,-331.481372,332.768955


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-994.119211,995.697386
6,-396.389280,398.246886
7,-62.858439,63.327789
8,-797.864784,798.057537
9,-518.841084,516.859235
10,-551.126279,551.876789
11,-124.631750,125.279408
12,-623.941045,625.843508
13,-320.675165,319.180355
14,-331.481372,332.768955


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-994.11921082,  995.69738557],
       [-396.38928026,  398.24688618],
       [ -62.85843901,   63.32778882],
       [-797.86478439,  798.05753671],
       [-518.84108427,  516.85923547],
       [-551.12627883,  551.876789  ],
       [-124.6317498 ,  125.2794077 ],
       [-623.94104465,  625.84350801],
       [-320.67516505,  319.18035549],
       [-331.48137178,  332.76895457]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1986.238422,-1991.394771
6,790.778561,-796.493772
7,123.716878,-126.655578
8,1593.729569,-1596.115073
9,1035.682169,-1033.718471
10,1100.252558,-1103.753578
11,247.263500,-250.558815
12,1245.882089,-1251.687016
13,639.350330,-638.360711
14,660.962744,-665.537909


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
